In [68]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab4").getOrCreate()
sc=spark.sparkContext

textFiles = sc.textFile('hdfs://localhost:9000//user/student/gutenberg/*')

textFiles.first()


'The Project Gutenberg eBook of Ulysses, by James Joyce'

In [69]:
words = textFiles\
    .flatMap(lambda line: line.split(" "))



counts = words\
    .map(lambda word: (word, 1))\
    .filter(lambda word: len(word[0]) >= 5)\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[1],word[0]))\
    .sortByKey(False)

counts.take(6)

[(3637, 'which'),
 (1691, 'their'),
 (1224, 'there'),
 (954, 'other'),
 (884, 'would'),
 (846, 'these')]

In [70]:
bigrams = textFiles.flatMap(lambda line: line.split("."))\
    .map(lambda line: line.strip().split(" "))\
    .flatMap(lambda words: (tuple(word) for word in zip(words, words[1: ])))

pair_counts = bigrams\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[1],word[0]))\
    .sortByKey(False)

pair_counts.take(7)

[(10929, ('', '')),
 (6900, ('of', 'the')),
 (3501, ('in', 'the')),
 (2208, ('to', 'the')),
 (1637, ('and', 'the')),
 (1618, ('on', 'the')),
 (1231, ('from', 'the'))]

In [71]:
bigrams1 = textFiles.flatMap(lambda line: line.split("-"))\
        .flatMap(lambda line: line.split(" "))\
        .flatMap(lambda line: line.split("—"))\


counts1 = words\
    .map(lambda word: (word, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[0]))\
    .sortBy(lambda word: len(word) * -1)

counts1.take(1)

['Nationalgymnasiummuseumsanatoriumandsuspensoriumsordinaryprivatdocentge']

In [72]:
bigrams2  = textFiles\
    .filter(lambda word: len(word) > 0 and word[0].isalpha())\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y)

counts2 = bigrams2.keys()\
    .map(lambda word: (word[0].upper(), 1)) \
    .reduceByKey(lambda x, y: x + y)\
    .sortByKey()

counts3.take(6)

[('A', 4943), ('B', 2990), ('C', 2675), ('D', 1843), ('E', 1405), ('F', 2148)]

In [79]:
import re
import sys

weatherFiles = sc.wholeTextFiles('hdfs://localhost:9000/user/student/weather/*')

def mapper(text):
    lines = text[1].split("\n")
    for line in lines:
        line = line.strip()
        (year, temp, q) = (line[15:19], line[87:92], line[92:93])
        if temp != "+9999" and re.match("[01459]", q):
            yield year, int(temp)
            
def reducer(year,temp):
    return max(year,temp)
                                     
weatherFiles\
    .flatMap(mapper)\
    .reduceByKey(reducer)\
    .collect()

23/01/07 01:03:23 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:191)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3033)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:829)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:754)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 312.0 failed 1 times, most recent failure: Lost task 0.0 in stage 312.0 (TID 871) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File

In [74]:
from pyspark.sql.functions import col
from pyspark.sql.functions import max

weatherLines = sc.textFile('hdfs://localhost:9000/user/student/weather/*')
weatherColumns = ["year","temperature", "quality"]
weatherDF = weatherLines\
    .map(lambda val: (int(val[15:19]), int(val[87:92]), int(val[92:93])))\
    .toDF(weatherColumns)

weatherDF.createOrReplaceTempView("Weather")

sqlWay = spark.sql("""
    SELECT year, max(temperature) FROM Weather
    WHERE quality IN (0,1,4,5,9) AND temperature != 9999
    GROUP BY year
""")

dataFrameWay = weatherDF\
    .where(weatherDF.quality.isin([0, 1, 4, 5, 9]) & (weatherDF.temperature != 9999))\
    .groupBy(weatherDF.year)\
    .agg(max(weatherDF.temperature))

print(sqlWay.rdd.map(lambda x: (x[0], x[1])).collect())
print(dataFrameWay.rdd.map(lambda x: (x[0], x[1])).collect())

[(2022, 320)]


[(2022, 320)]
